In [1]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool


# 步骤 1（可选）：添加一个名为 `my_image_gen` 的自定义工具。
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    # `description` 用于告诉智能体该工具的功能。
    description = 'AI 绘画（图像生成）服务，输入文本描述，返回基于文本信息绘制的图像 URL。'
    # `parameters` 告诉智能体该工具有哪些输入参数。
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': '期望的图像内容的详细描述',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # `params` 是由 LLM 智能体生成的参数。
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)


# 步骤 2：配置您所使用的 LLM。
llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'qwen-max',
    'model_server': 'dashscope',
    'api_key': 'sk-882e296067b744289acf27e6e20f3ec0',
    # 如果这里没有设置 'api_key'，它将读取 `DASHSCOPE_API_KEY` 环境变量。

    # 使用与 OpenAI API 兼容的模型服务，例如 vLLM 或 Ollama：
    # 'model': 'Qwen2-7B-Chat',
    # 'model_server': 'http://localhost:8000/v1',  # base_url，也称为 api_base
    # 'api_key': 'EMPTY',
    # （可选） LLM 的超参数：
    'generate_cfg': {
        'top_p': 0.8
    }
}

# 步骤 3：创建一个智能体。这里我们以 `Assistant` 智能体为例，它能够使用工具并读取文件。
system_instruction = '''你是一个乐于助人的AI助手。
在收到用户的请求后，你应该：
- 首先绘制一幅图像，得到图像的url，
- 然后运行代码`request.get`以下载该图像的url，
- 最后从给定的文档中选择一个图像操作进行图像处理。
用 `plt.show()` 展示图像。
你总是用中文回复用户。'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter` 是框架自带的工具，用于执行代码。
#files = ['./examples/resource/doc.pdf']  # 给智能体一个 PDF 文件阅读。
files = ['./docs/平安商业综合责任保险（亚马逊）.pdf']  # 给智能体一个 PDF 文件阅读。
bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                files=files)

# 步骤 4：作为聊天机器人运行智能体。
messages = []  # 这里储存聊天历史。
while True:
    # 例如，输入请求 "绘制一只狗并将其旋转 90 度"。
    query = input('用户请求: ')
    if query == '-1':
        break
    # 将用户请求添加到聊天历史。
    messages.append({'role': 'user', 'content': query})
    response = []
    current_index = 0
    for response in bot.run(messages=messages):
        # 流式输出。
        #print('机器人回应:')
        #pprint.pprint(response, indent=2)
        current_response = response[0]['content'][current_index:]
        current_index = len(response[0]['content'])
        print(current_response, end='')
    # 将机器人的回应添加到聊天历史。
    messages.extend(response)

用户请求:  平安商业综合责任保险介绍下


2025-03-10 20:11:44,641 - memory.py - 113 - INFO - {"keywords_zh": ["平安", "商业", "综合责任保险", "介绍"], "keywords_en": ["Ping An", "commercial", "comprehensive liability insurance", "introduction"], "text": "平安商业综合责任保险介绍下"}
2025-03-10 20:11:45,520 - simple_doc_parser.py - 382 - INFO - Start parsing ./docs/平安商业综合责任保险（亚马逊）.pdf...
2025-03-10 20:11:45,546 - simple_doc_parser.py - 425 - INFO - Finished parsing ./docs/平安商业综合责任保险（亚马逊）.pdf. Time spent: 0.02223944664001465 seconds.
2025-03-10 20:11:45,556 - doc_parser.py - 108 - INFO - Start chunking ./docs/平安商业综合责任保险（亚马逊）.pdf (平安商业综合责任保险（亚马逊）.pdf)...
2025-03-10 20:11:45,559 - doc_parser.py - 126 - INFO - Finished chunking ./docs/平安商业综合责任保险（亚马逊）.pdf (平安商业综合责任保险（亚马逊）.pdf). Time spent: 0.0 seconds.
2025-03-10 20:11:45,566 - base_search.py - 56 - INFO - all tokens: 0
2025-03-10 20:11:45,568 - base_search.py - 59 - INFO - use full ref


您所提到的“平安商业综合责任保险（亚马逊）”似乎指向一个特定的产品文件，但是提供的文档内容为空，没有具体的条款或细节。通常来说，平安商业综合责任保险是一种为企业提供保障的保险产品，它覆盖了企业在经营过程中可能遇到的各种责任风险。

为了更准确地介绍这个保险产品，我需要根据实际的保险条款来说明其特点、保障范围、除外责任等信息。如果可以，请您上传含有具体内容的文档或者告诉我更多关于该保险产品的详细信息，这样我才能够为您提供更为详尽和准确的介绍。

如果您有其他问题或需要了解一般性的商业综合责任保险的信息，也请随时告知。

用户请求:  -1
